In [1]:
import gamspy as gp
import gamspy.math as gpm
from gamspy import Sum, Card, Container, Options, Problem, Sense

import sys
import numpy as np
import pandas as pd
import math
import itertools

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import contextily as ctx

options = Options(equation_listing_limit=0)
m = Container(options=options,debugging_level='keep')

In [2]:
drone = pd.read_csv("/home/samjenkins2001/CS524/Project/Optimize/drone.csv")
loc = pd.read_csv("/home/samjenkins2001/CS524/Project/Optimize/location.csv")

drone_df = pd.DataFrame(drone)
loc_df = pd.DataFrame(loc)

In [3]:
locations = list(loc_df["Location"])
models = list(drone_df["Drone Models"])
loc_df.index = locations
drone_df.index = models
loc_df = loc_df.drop(columns = "Location")
drone_df = drone_df.drop(columns = "Drone Models")
display(loc_df)
display(drone_df)

,Latitude,Longitude,Payload,Demand
Camp Randall,43.070227,-89.412687,5.0000,5 Footballs
Sam's Apartment,43.075462,-89.393407,2.6250,History of Optimization book
3006 Sunrise Ct,43.108656,-89.473928,5.0000,Bird Food
Kohl Center,43.069709,-89.396907,4.6875,5 New Basketball Jerseys
Computer Sciences Building,43.071286,-89.406561,4.9360,8 NVIDIA Jetsons
Noland Hall,43.071678,-89.404489,1.5000,4 cups of Pepper Seeds
Lake Mendota,43.107811,-89.419515,4.0000,Fishing Rod
The Kollege Klub,43.075951,-89.397279,3.7500,Bottle of NA Vodka
Tenney Park,43.092186,-89.367120,2.2000,Drinking Fountain Filter
The Bakke,43.076852,-89.420159,3.4500,5 BUCKS Pre Workout


,Carrying Capacity,Fly in Rain,Speed,Radius,Charging Current
MK30,5,1,80.47,32.1,840
MK27,5,0,80.47,16.1,560


In [4]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth's radius in km
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    
    a = math.sin(delta_phi/2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    
    return R * c

# Define depot and charging station
depot = {"DWI 4": (43.103263, -89.323598)}
sellery_location = [43.075182, -89.400503]

# Consumer locations with payload weights
locations = {
    "Camp Randall": (43.070227, -89.412687, 5.0),
    "Sam's Apartment": (43.075462, -89.393407, 2.6250),
    "3006 Sunrise Ct": (43.108656, -89.473928, 5.0000),
    "Kohl Center": (43.069709, -89.396907, 4.6875),
    "Computer Sciences Building": (43.071286, -89.406561, 4.936),
    "Noland Hall": (43.071678, -89.404489, 1.5000),
    "Lake Mendota": (43.107811, -89.419515, 4.0000),
    "The Kollege Klub": (43.075951, -89.397279, 3.75),
    "Tenney Park": (43.092186, -89.367120, 2.2000),
    "The Bakke": (43.076852, -89.420159, 3.4500),
    "The Nick": (43.070881, -89.399039, 4.16),
    "Governer's Mansion": (43.113096, -89.370588, 0.7000),
    "Wisconsin State Capital": (43.076583, -89.384819, 5.0000),
    "Henry Vilas Zoo": (43.060791, -89.410046, 3.0000),
    "The Edgewater": (43.079474, -89.389944, 3.4000),
    "Nandini's Apartment": (43.069867, -89.394065, 2.6250),
    "Sellery": (43.075182, -89.400503, 0),
    "DWI 4": (43.103263, -89.323598, 0)
}

# Max initial ranges
initial_range_mk27 = 16.1
initial_range_mk30 = 32.1


In [5]:
def calculate_distance_matrix(locations):
    location_names = list(locations.keys())
    distance_matrix = pd.DataFrame(index=location_names, columns=location_names, dtype=float)
    
    # Fill the DataFrame with distances
    for loc1, loc2 in itertools.combinations(location_names, 2):
        lat1, lon1 = locations[loc1][:2]
        lat2, lon2 = locations[loc2][:2]
        distance = haversine(lat1, lon1, lat2, lon2)
        
        # Assign distances symmetrically
        distance_matrix.loc[loc1, loc2] = distance
        distance_matrix.loc[loc2, loc1] = distance
        
    # Fill diagonal with zeros (distance from a location to itself)
    distance_matrix.fillna(0, inplace=True)
    
    return distance_matrix

distance_matrix = calculate_distance_matrix(locations)
display(distance_matrix)

,Camp Randall,Sam's Apartment,3006 Sunrise Ct,Kohl Center,Computer Sciences Building,Noland Hall,Lake Mendota,The Kollege Klub,Tenney Park,The Bakke,The Nick,Governer's Mansion,Wisconsin State Capital,Henry Vilas Zoo,The Edgewater,Nandini's Apartment,Sellery,DWI 4
Camp Randall,0.000000,1.670730,6.556717,1.283105,0.511353,0.685181,4.215772,1.404078,4.433676,0.954476,1.111001,5.865889,2.371363,1.070945,2.114153,1.513196,1.132698,8.113906
Sam's Apartment,1.670730,0.000000,7.508253,0.700032,1.164985,0.993626,4.175304,0.319160,2.831189,2.178340,0.684650,4.576557,0.708586,2.118531,0.527378,0.624427,0.577199,6.456905
3006 Sunrise Ct,6.556717,7.508253,0.000000,7.607455,6.869697,6.978635,4.418215,7.208503,8.862993,5.618606,7.390887,8.403234,8.066826,7.432470,7.552009,7.788551,7.028655,12.218878
Kohl Center,1.283105,0.700032,7.607455,0.000000,0.803556,0.653639,4.617416,0.694736,3.478365,2.048878,0.216738,5.276588,1.244301,1.456918,1.224281,0.231524,0.675037,7.025834
Computer Sciences Building,0.511353,1.164985,6.869697,0.803556,0.000000,0.173857,4.195409,0.915145,3.957428,1.266079,0.612659,5.490554,1.861625,1.200840,1.628062,1.027232,0.655595,7.617916
Noland Hall,0.685181,0.993626,6.978635,0.653639,0.173857,0.000000,4.198998,0.754135,3.796150,1.396776,0.451478,5.365458,1.688219,1.292008,1.465313,0.870349,0.506590,7.448851
Lake Mendota,4.215772,4.175304,4.418215,4.617416,4.195409,4.198998,0.000000,3.976262,4.595092,3.442881,4.430290,4.014962,4.471569,5.284634,3.961538,4.698150,3.942971,7.803234
The Kollege Klub,1.404078,0.319160,7.208503,0.694736,0.915145,0.754135,3.976262,0.000000,3.042661,1.861049,0.581601,4.664400,1.014461,1.979191,0.713005,0.725134,0.275470,6.709828
Tenney Park,4.433676,2.831189,8.862993,3.478365,3.957428,3.796150,4.595092,3.042661,0.000000,4.632542,3.511723,2.342071,2.253025,4.933824,2.330995,3.308778,3.305303,3.742207
The Bakke,0.954476,2.178340,5.618606,2.048878,1.266079,1.396776,3.442881,1.861049,4.632542,0.000000,1.839474,5.695863,2.870513,1.965782,2.471299,2.257332,1.607266,8.373013


In [6]:
def get_possible_routes():
    possible_routes = []
    unique_routes_set = set()  # Track unique routes by their items
    
    consumers = list(locations.keys())[:-2]
    
    # Generate all subsets of consumers
    for r in range(1, len(consumers) + 1):
        for subset in itertools.combinations(consumers, r):
            total_payload = sum(locations[consumer][2] for consumer in subset)
            
            # Skip if total payload exceeds 5 lbs
            if total_payload > 5:
                continue
            
            # Create a sorted tuple of consumers for route uniqueness
            sorted_route = tuple(sorted(subset))
            if sorted_route in unique_routes_set:
                continue
            unique_routes_set.add(sorted_route)
            
            # Distance calculation: Depot -> Consumers -> Depot (checking if recharge is needed)
            total_distance = 0
            start_lat, start_lon = depot["DWI 4"]
            route_with_recharge = ["DWI 4"]  # Start with DWI 4

            for consumer in subset:
                consumer_lat, consumer_lon = locations[consumer][:2]
                total_distance += haversine(start_lat, start_lon, consumer_lat, consumer_lon)
                start_lat, start_lon = consumer_lat, consumer_lon
                route_with_recharge.append(consumer)  # Add consumer name, not coordinates

            # Check if the total distance exceeds the range, and if so, add Sellery for recharge
            adjusted_range_mk30 = initial_range_mk30 * (1 - total_payload / 100)
            adjusted_range_mk27 = initial_range_mk27 * (1 - total_payload / 100)
            
            if total_distance > adjusted_range_mk30:  # Check for MK30 range
                # Add Sellery as a recharge stop
                route_with_recharge.append("Sellery")  # Add Sellery name as a stop
                total_distance += haversine(start_lat, start_lon, sellery_location[0], sellery_location[1])
                start_lat, start_lon = sellery_location  # Update to Sellery location

                # Recharge at Sellery (reset distance after recharge)
                total_distance = 0  # Reset total distance after recharge

            # Return to depot after all consumers and possible recharge
            total_distance += haversine(start_lat, start_lon, depot["DWI 4"][0], depot["DWI 4"][1])

            route_with_recharge.append("DWI 4")

            # Determine feasibility based on the adjusted range
            mk30_feasible = total_distance <= adjusted_range_mk30
            mk27_feasible = total_distance <= adjusted_range_mk27
            
            drone_type = None
            if mk30_feasible and mk27_feasible:
                drone_type = "Both"
            elif mk30_feasible:
                drone_type = "MK30"
            elif mk27_feasible:
                drone_type = "MK27"
            
            if drone_type:
                possible_routes.append({
                    "Route": tuple(route_with_recharge),  # Store as a tuple
                    "Payload (lbs)": total_payload,
                    "Total Distance (km)": total_distance,
                    "Drone Type": drone_type
                })
    
    return possible_routes


# Get all possible routes
routes = get_possible_routes()

# Convert to DataFrame and display
routes_df = pd.DataFrame(routes)
routes_df.to_csv('routes.csv', index=False)
display(routes_df)


,Route,Payload (lbs),Total Distance (km),Drone Type
0,"(DWI 4, Camp Randall, DWI 4)",5.0000,16.227811,MK30
1,"(DWI 4, Sam's Apartment, DWI 4)",2.6250,12.913810,Both
2,"(DWI 4, 3006 Sunrise Ct, DWI 4)",5.0000,24.437757,MK30
3,"(DWI 4, Kohl Center, DWI 4)",4.6875,14.051669,Both
4,"(DWI 4, Computer Sciences Building, DWI 4)",4.9360,15.235831,Both
5,"(DWI 4, Noland Hall, DWI 4)",1.5000,14.897702,Both
6,"(DWI 4, Lake Mendota, DWI 4)",4.0000,15.606467,MK30
7,"(DWI 4, The Kollege Klub, DWI 4)",3.7500,13.419655,Both
8,"(DWI 4, Tenney Park, DWI 4)",2.2000,7.484415,Both
9,"(DWI 4, The Bakke, DWI 4)",3.4500,16.746026,MK30


In [11]:
# Mapping location names to integers
location_to_int = {
    "DWI 4": 1,
    "Camp Randall": 2,
    "Sam's Apartment": 3,
    "3006 Sunrise Ct": 4,
    "Kohl Center": 5,
    "Computer Sciences Building": 6,
    "Noland Hall": 7,
    "Lake Mendota": 8,
    "The Kollege Klub": 9,
    "Tenney Park": 10,
    "The Bakke": 11,
    "The Nick": 12,
    "Governer's Mansion": 13,
    "Wisconsin State Capital": 14,
    "Henry Vilas Zoo": 15,
    "The Edgewater": 16,
    "Nandini's Apartment": 17,
    "Sellery": 18,
}

# Generate tuples of customers (as integers) and their corresponding routes
def generate_customer_route_tuples(routes_df):
    customer_route_tuples = []

    # Iterate over each row in the DataFrame
    for route_index, route in routes_df.iterrows():
        route_customers = route['Route']  # Get the list of customer names for this route
        
        # Initialize a flag to check if DWI 4 (18) has been added
        dwi4_added = False

        for customer in route_customers:
            customer_int = location_to_int[customer]  # Map customer name to integer
            
            # Only add DWI 4 once for the route
            if customer_int == 1 and not dwi4_added:
                customer_route_tuples.append((str(customer_int), f'r{route_index + 1}'))  # 1-based route index
                dwi4_added = True
            elif customer_int != 1:
                customer_route_tuples.append((str(customer_int), f'r{route_index + 1}'))  # 1-based route index

    return customer_route_tuples



# Generate the customer-route tuples
location_route_tuples = generate_customer_route_tuples(routes_df)
customer_route_tuples = [tuple for tuple in location_route_tuples if tuple[0] != '1']
distance_list = routes_df["Total Distance (km)"].tolist()
payload_list = loc_df["Payload"].tolist()

In [ ]:
#CHANGE SO THAT THE DRONES ARE IN 1 SET
#ADD WEATHER CONSTRAINT AFTER, ONLY MK30's CAN WORK IN THE RAIN

k = m.addSet('k', records = [i+1 for i in range(10)], description='Amount of mk30 Drones in stock')
# mk27 = m.addSet('mk27', records = [i+1 for i in range(20)], description='Amount of mk27 Drones in stock')
i = m.addSet('i', records=[i+1 for i in range(18)], description='All Locations')
z = m.addAlias('z', i)
s = m.addSet('s',records=[i+2 for i in range(16)], description='Consumers')
j = m.addSet('j',records=['r'+str(i+1) for i in range(37)], description='Feasible Routes')
L = m.addSet('L',[i,j], records = location_route_tuples, description="Data showing which locations are covered by which routes")
C = m.addSet('C',[s,j], records = customer_route_tuples, description="Data showing which customers are covered by which routes")

max_battery = m.addParameter("max_battery", records=100)
distance = m.addParameter("distance", domain=[i, z], records = np.array(distance_matrix), description= "Distance for feasible route")
mk30_range = m.addParameter("mk30_range", records=32.1, description="Range of MK30 Drone Fully Charged (km)")
mk27_range = m.addParameter("mk27_range", records=16.1, description="Range of MK27 Drone Fully Charged (km)")
payload = m.addParameter("payload", domain=[i], records=np.array(payload_list), description="Payload for Route i")
alpha = m.addParameter("alpha", records=0.02, description="Constant for battery drained with respect to payload")

x = m.addVariable('x','binary', domain=[j, k], description = "Binary Variable denoting if route j is taken by drone k")
y = m.addVariable('y','binary', domain=[i, k], description = "Binary Variable denoting if location i is visited by drone k")
b = m.addVariable('b', 'positive', domain=[i, k], description="MK30 Battery Level at location i")
delta = m.addVariable('delta', 'positive', domain=[k], description="Charge added to drone k at Sellery")

location_route_link = m.addEquation('location_route_link', domain=[i, k], description="Link locations to routes for each drone")
location_route_link[i, k] = y[i, k] == Sum(L[i, j], x[j, k])

cover = m.addEquation('cover',domain=[s, k], description="Each Delivery is Made Once")
cover[s, k]= Sum(C[s,j], x[j, k]) == 1

# Must find payload between for each individual delivery
battery_consumption = m.addEquation('battery_consumption', domain=[i, z, k], description="Ensure Battery Drains Accurately")
battery_consumption[i, z, k] = b[z, k] == b[i, k] - (distance[i, z] / mk30_range + alpha * payload[i]) * y[i, k]

#18 is our recharge location
recharging = m.addEquation('recharging', domain=[k], description="Ensure Charge doesn't exceed 100")
recharging[k] = b["18", k] + delta[k] <= 100

positive_battery = m.addEquation('positive_battery', domain=[i, k], description="Battery level stays Non-Negative")
positive_battery[i, k] = b[i, k] >= 0

#18 is the depot
battery_init = m.addEquation('battery_init', domain=[k], description="Battery starts full")
battery_init[k] = b["1", k] == 100


setcover = m.addModel('setcover',
    equations=m.getEquations(),
    problem=Problem.MIP,
    sense=Sense.MIN,
    objective=Sum([j, k], x[j, k]),
)

setcover.solve(output=None,options=Options(absolute_optimality_gap=0.999))
# x.records.to_csv('optimal.csv', index=False)
# display(x.records)

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,IntegerInfeasible,NA,3761,741,MIP,CPLEX,0.004


In [14]:
# Pareto Curve showing Tradeoff between Energy Consumption and Delivery Time


In [10]:
# Extract the selected routes (those with x[j] == 1)
selected_routes = [j for j in range(len(x.records)) if x.records.level[j] == 1.0]

# Coordinates for delivery locations (same as before)
# Coordinates for delivery locations (same as before)
latitudes = list(loc_df["Latitude"])
longitudes = list(loc_df["Longitude"])
coordinates = list(zip(latitudes, longitudes))

delivery_coords = coordinates[:-2]
charge_coords = coordinates[-2:]

# Set up the map
fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={'projection': ccrs.PlateCarree()})
ax.set_extent([-89.477, -89.32, 43.05, 43.15], crs=ccrs.PlateCarree())
ctx.add_basemap(ax, crs=ccrs.PlateCarree(), source=ctx.providers.OpenStreetMap.Mapnik)

# Plot delivery locations
for coord in delivery_coords:
    latitude = coord[0]
    longitude = coord[1]
    ax.plot(longitude, latitude, marker='o', color='blue', markersize=4, transform=ccrs.PlateCarree(), label="Delivery Location" if coord == delivery_coords[0] else "")

# Plot charge locations
for coord in charge_coords:
    latitude = coord[0]
    longitude = coord[1]
    ax.plot(longitude, latitude, marker='o', color='red', markersize=4 if coord == charge_coords[0] else 10, transform=ccrs.PlateCarree(), label="Charging Location" if coord == charge_coords[0] else "Depot")


depot_lat, depot_lon = charge_coords[1]  # Assuming the second charge location is the Depot

# Iterate over each selected route index
for route_index in selected_routes:
    # Get the list of customers for this route from your customer_route_tuples
    customers_in_route = [customer for customer, route in customer_route_tuples if route == f'r{route_index+1}']
    
    # Plot dashed lines from the Depot to each customer in the route
    previous_customer_coords = (depot_lat, depot_lon)  # Start with the Depot
    for customer_index in customers_in_route:
        # Get the customer's coordinates from delivery_coords (mapping customer to delivery point)
        customer_lat, customer_lon = delivery_coords[int(customer_index) - 1]  # Adjust for 0-indexing
        
        # Plot a dashed line from the previous customer (or Depot) to the current customer
        ax.plot([previous_customer_coords[1], customer_lon], 
                [previous_customer_coords[0], customer_lat], 
                color='black', linestyle='--', linewidth=1, transform=ccrs.PlateCarree(), label="Route" if (route_index == selected_routes[0] and previous_customer_coords == (depot_lat, depot_lon)) else "")
        
        # Update the previous customer coordinates
        previous_customer_coords = (customer_lat, customer_lon)

    # Return to the Depot after the last delivery
    ax.plot([previous_customer_coords[1], depot_lon], 
            [previous_customer_coords[0], depot_lat], 
            color='black', linestyle='--', linewidth=1, transform=ccrs.PlateCarree())

# Add the legend
plt.legend()

# Add a title
plt.title("Map of Madison, WI with Delivery Locations and Routes (OpenStreetMap)")

# Show the map
plt.show()


TypeError: object of type 'NoneType' has no len()